# Loading Image

We use this section to load an image and preprocess it. 

In [1]:
# Imports for visualization
import cv2
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats
import pyautogui

In [2]:
def display_image(img):
    #imS = cv2.resize(img, (1000, 700))  # Resize image
    imS = cv2.resize(img, (800, 800))
    cv2.imshow("image", imS)
    cv2.waitKey(0)

    cv2.waitKey(0)
    cv2.destroyAllWindows()
    

def make_kernel(a):
  """Transform a 2D array into a convolution kernel"""
  a = np.asarray(a)
  a = a.reshape(list(a.shape) + [1,1])
  return a

In [3]:
def take_screenshot():
    myScreenshot = pyautogui.screenshot(region=(670,165, 695, 700))
    myScreenshot.save(r'screenshot1.png')
    return r'screenshot1.png'

In [4]:
import tkinter as tk
from tkinter import filedialog

root= tk.Tk()

canvas1 = tk.Canvas(root, width = 300, height = 300)
canvas1.pack()

def takeScreenshot ():
    
    myScreenshot = pyautogui.screenshot()
    file_path = filedialog.asksaveasfilename(defaultextension='.png')
    myScreenshot.save(file_path)

myButton = tk.Button(text='Take Screenshot', command=take_screenshot, bg='green',fg='white',font= 10)
canvas1.create_window(150, 150, window=myButton)

root.mainloop()

In [5]:
#Load image and convert to grayscale
image_path = 'screenshot1.png'
img = cv2.imread(image_path)

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

#Perform Hough transform
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray,50,150,apertureSize = 3)
lines = cv2.HoughLines(edges,1,np.pi/180,200)



In [6]:
vertical_lines = []
horizontal_lines = []
#Display all lines
for line in lines:
    rho = line[0][0]
    theta = line[0][1]
    if(theta == 0):
        vertical_lines.append(line)



#Display all lines
for line in lines:
    rho = line[0][0]
    theta = line[0][1]
    if theta != 0.0:        
        horizontal_lines.append(line)
        

In [7]:
def checkMatch(lineset):
    """Checks whether there exists 7 lines of consistent increasing order in set of lines"""
    linediff = np.diff(lineset)
    x = 0
    cnt = 0
    for line in linediff:
        # Within 5 px of the other (allowing for minor image errors)
        if np.abs(line - x) < 5:
            cnt += 1
        else:
            cnt = 0
            x = line
    return cnt == 5

In [8]:
len(horizontal_lines)

13

In [9]:
img = cv2.imread(image_path)
for line in vertical_lines:
    rho = line[0][0]
    theta = line[0][1]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 5000*(-b))
    y1 = int(y0 + 5000*(a))
    x2 = int(x0 - 5000*(-b))
    y2 = int(y0 - 5000*(a))

    cv2.line(img,(x1,y1),(x2,y2),(0,0,255),2)

display_image(img)

In [10]:
img = cv2.imread(image_path)
for line in horizontal_lines:
    rho = line[0][0]
    theta = line[0][1]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 5000*(-b))
    y1 = int(y0 + 5000*(a))
    x2 = int(x0 - 5000*(-b))
    y2 = int(y0 - 5000*(a))

    cv2.line(img,(x1,y1),(x2,y2),(0,0,255),2)

display_image(img)

In [11]:

#Convert to numpy arrays. 
horizontal_lines_np = np.concatenate( horizontal_lines, axis=0 )
vertical_lines_np = np.concatenate( vertical_lines, axis=0 )
horizontal_lines_np = np.sort(horizontal_lines_np, axis = 0)
vertical_lines_np = np.sort(vertical_lines_np, axis = 0)

horizontal_lines_np

array([[  2.       ,   1.5707964],
       [ 75.       ,   1.5707964],
       [ 78.       ,   1.5707964],
       [ 88.       ,   1.5707964],
       [174.       ,   1.5707964],
       [251.       ,   1.5707964],
       [260.       ,   1.5707964],
       [346.       ,   1.5707964],
       [432.       ,   1.5707964],
       [518.       ,   1.5707964],
       [595.       ,   1.5707964],
       [604.       ,   1.5707964],
       [691.       ,   1.5707964]], dtype=float32)

In [12]:
linediff_hor = np.diff(horizontal_lines_np, axis = 0)
linediff_ver = np.diff(vertical_lines_np, axis = 0)

linediff_hor

array([[73.,  0.],
       [ 3.,  0.],
       [10.,  0.],
       [86.,  0.],
       [77.,  0.],
       [ 9.,  0.],
       [86.,  0.],
       [86.,  0.],
       [86.,  0.],
       [77.,  0.],
       [ 9.,  0.],
       [87.,  0.]], dtype=float32)

In [13]:
#Remove lines which are very close to each other so that the board detection will be easier. 
def filter_lines(lines, linediff):
    indices_to_delete = []
    for index, diff in enumerate(linediff): 
        if(diff[0] <= 10):
            indices_to_delete.append(index)
    return np.delete(lines, indices_to_delete,0)


In [14]:
filtered_horizontal_lines = filter_lines(horizontal_lines_np, linediff_hor)
filtered_vertical_lines = filter_lines(vertical_lines_np, linediff_ver)

In [15]:
img = cv2.imread(image_path)
for line in filtered_horizontal_lines:
    rho = line[0]
    theta = line[1]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 5000*(-b))
    y1 = int(y0 + 5000*(a))
    x2 = int(x0 - 5000*(-b))
    y2 = int(y0 - 5000*(a))

    cv2.line(img,(x1,y1),(x2,y2),(0,0,255),2)

display_image(img)

In [16]:
#update line differences. 

linediff_hor = np.diff(filtered_horizontal_lines, axis = 0)
linediff_ver = np.diff(filtered_vertical_lines, axis = 0)

In [17]:
linediff_hor

array([[86.,  0.],
       [86.,  0.],
       [86.,  0.],
       [86.,  0.],
       [86.,  0.],
       [86.,  0.],
       [86.,  0.],
       [87.,  0.]], dtype=float32)

In [18]:
def get_chess_lines(lines, linediff):
    indices_to_delete = []
    #find most occuring difference which is probably the width of the chess square. 
    m = stats.mode(linediff)
    mode = m[0][0][0]
    for index, diff in enumerate(linediff): 
        if(abs(diff[0] - mode) >5):
            print(diff[0])
            indices_to_delete.append(index+1)
    return np.delete(lines, indices_to_delete,0)

In [19]:
chess_lines_hor = get_chess_lines(filtered_horizontal_lines, linediff_hor)
chess_lines_ver = get_chess_lines(filtered_vertical_lines, linediff_ver)

In [20]:
img = cv2.imread(image_path)
for line in chess_lines_hor:
    rho = line[0]
    theta = line[1]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 5000*(-b))
    y1 = int(y0 + 5000*(a))
    x2 = int(x0 - 5000*(-b))
    y2 = int(y0 - 5000*(a))

    cv2.line(img,(x1,y1),(x2,y2),(0,0,255),2)
    
for line in chess_lines_ver:
    rho = line[0]
    theta = line[1]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 5000*(-b))
    y1 = int(y0 + 5000*(a))
    x2 = int(x0 - 5000*(-b))
    y2 = int(y0 - 5000*(a))

    cv2.line(img,(x1,y1),(x2,y2),(0,0,255),2)

print(chess_lines_hor)
    
display_image(img)

[[  2.          1.5707964]
 [ 88.          1.5707964]
 [174.          1.5707964]
 [260.          1.5707964]
 [346.          1.5707964]
 [432.          1.5707964]
 [518.          1.5707964]
 [604.          1.5707964]
 [691.          1.5707964]]


In [21]:
#Get equations of lines:
def getIntersection(rho1,theta1,rho2, theta2):
    A = np.array([
        [np.cos(theta1), np.sin(theta1)],
        [np.cos(theta2), np.sin(theta2)]
    ])
    b = np.array([[rho1], [rho2]])
    x0, y0 = np.linalg.solve(A, b)
    x0, y0 = int(np.round(x0)), int(np.round(y0))
    return [[x0, y0]]
    

In [22]:
#Get all intersection points of horizontal and vertical lines to get the corners of the chess squares to extract patches. 

points = []
for i in chess_lines_hor:
    for j in chess_lines_ver:
        rho_hor = i[0]
        theta_hor = i[1]
        rho_ver = j[0]
        theta_ver = j[1]
        points.append(getIntersection(rho_hor, theta_hor, rho_ver, theta_ver)[0])

In [23]:
# Make empty black image
img = cv2.imread(image_path)
for point in points:
    print(point)
    cv2.circle(img, (point[0],point[1]), 5, [0,0,255], thickness=2, lineType=8, shift=0)
#image[10,5]=[0,0,255]

display_image(img)
# # Make one pixel red

# # Save
# cv2.imwrite("result.png",image)

[1, 2]
[87, 2]
[173, 2]
[259, 2]
[345, 2]
[431, 2]
[517, 2]
[603, 2]
[690, 2]
[1, 88]
[87, 88]
[173, 88]
[259, 88]
[345, 88]
[431, 88]
[517, 88]
[603, 88]
[690, 88]
[1, 174]
[87, 174]
[173, 174]
[259, 174]
[345, 174]
[431, 174]
[517, 174]
[603, 174]
[690, 174]
[1, 260]
[87, 260]
[173, 260]
[259, 260]
[345, 260]
[431, 260]
[517, 260]
[603, 260]
[690, 260]
[1, 346]
[87, 346]
[173, 346]
[259, 346]
[345, 346]
[431, 346]
[517, 346]
[603, 346]
[690, 346]
[1, 432]
[87, 432]
[173, 432]
[259, 432]
[345, 432]
[431, 432]
[517, 432]
[603, 432]
[690, 432]
[1, 518]
[87, 518]
[173, 518]
[259, 518]
[345, 518]
[431, 518]
[517, 518]
[603, 518]
[690, 518]
[1, 604]
[87, 604]
[173, 604]
[259, 604]
[345, 604]
[431, 604]
[517, 604]
[603, 604]
[690, 604]
[1, 691]
[87, 691]
[173, 691]
[259, 691]
[345, 691]
[431, 691]
[517, 691]
[603, 691]
[690, 691]


In [24]:
# create a 2D list for easier point arrangement
new_points = []
print(len(points))

if(len(points) != 81) :
    print("Board not detected! Try again!")
else:
    for i in range(0,81):
        if(i % 9) == 0:
            new_points.append([])
        new_points[int(i/9)].append(points[i])

81


In [25]:
new_points

[[[1, 2],
  [87, 2],
  [173, 2],
  [259, 2],
  [345, 2],
  [431, 2],
  [517, 2],
  [603, 2],
  [690, 2]],
 [[1, 88],
  [87, 88],
  [173, 88],
  [259, 88],
  [345, 88],
  [431, 88],
  [517, 88],
  [603, 88],
  [690, 88]],
 [[1, 174],
  [87, 174],
  [173, 174],
  [259, 174],
  [345, 174],
  [431, 174],
  [517, 174],
  [603, 174],
  [690, 174]],
 [[1, 260],
  [87, 260],
  [173, 260],
  [259, 260],
  [345, 260],
  [431, 260],
  [517, 260],
  [603, 260],
  [690, 260]],
 [[1, 346],
  [87, 346],
  [173, 346],
  [259, 346],
  [345, 346],
  [431, 346],
  [517, 346],
  [603, 346],
  [690, 346]],
 [[1, 432],
  [87, 432],
  [173, 432],
  [259, 432],
  [345, 432],
  [431, 432],
  [517, 432],
  [603, 432],
  [690, 432]],
 [[1, 518],
  [87, 518],
  [173, 518],
  [259, 518],
  [345, 518],
  [431, 518],
  [517, 518],
  [603, 518],
  [690, 518]],
 [[1, 604],
  [87, 604],
  [173, 604],
  [259, 604],
  [345, 604],
  [431, 604],
  [517, 604],
  [603, 604],
  [690, 604]],
 [[1, 691],
  [87, 691],
  [173, 69

In [26]:
def get_square_coordinates(points):
    squares = []
    for i in range(0, 8):
        for j in range(0,8):
            squares.append([(i,j), (i, j + 1), (i + 1, j), (i + 1, j + 1)])
    return squares

def get_ith_square(i, points, squares):
    square = squares[i-1]
    return [points[square[0][0]][square[0][1]], points[square[1][0]][square[1][1]],
            points[square[2][0]][square[2][1]],points[square[3][0]][square[3][1]]]

In [27]:
squares = get_square_coordinates(new_points)


In [28]:
img = cv2.imread(image_path)

for i in range(1,10):
    corners = get_ith_square(i, new_points, squares)
    img_patch = img[corners[0][0] : corners[3][0], corners[0][1] : corners[3][1]]
    display_image(cv2.cvtColor(img_patch, cv2.COLOR_BGR2GRAY))


In [29]:
import pickle
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import tensorflow as tf
from keras.models import load_model

model = load_model('model.h5', custom_objects={'softmax_v2': tf.nn.softmax})


Using TensorFlow backend.


In [35]:
def decode(s):
    if s == 'wr':
        return 0
    if s == 'wh':
        return 1
    if s == 'wb':
        return 2
    if s == 'wq':
        return 3
    if s == 'wk':
        return 4
    if s == 'wp':
        return 5
    if s == 'br':
        return 6
    if s == 'bh':
        return 7
    if s == 'bb':
        return 8
    if s == 'bq':
        return 9
    if s == 'bk':
        return 10
    if s == 'bp':
        return 11
    if s == 'n':
        return 12
    
def encode(s):
    s = s[0]
    if s == 0:
        return "White Rook"
    if s == 1:
        return "White Horse"
    if s == 2:
        return "White Bishop"
    if s == 3:
        return "White Queen"
    if s == 4:
        return "White King"
    if s == 5:
        return "White Pawn"
    if s == 6:
        return "Black Rook"
    if s == 7:
        return "Black Horse"
    if s == 8:
        return "Black Bishop"
    if s == 9:
        return "Black Queen"
    if s == 10:
        return "Black King"
    if s == 11:
        return "Black Pawn"
    if s == 12:
        return "Blank Square"

In [37]:
prediction = []
l1= []
l2 = []
for i in range(1,65):
    corners = get_ith_square(i, new_points, squares)
    img_patch = img[corners[0][0] : corners[3][0], corners[0][1] : corners[3][1]]
    img_patch_short = cv2.resize(cv2.cvtColor(img_patch, cv2.COLOR_BGR2GRAY),(28,28))
    
    temp_lst = []

    temp_lst.append(img_patch_short)
    inp = np.dstack(temp_lst)

    inp = np.swapaxes(inp, 0, 2)
    inp = inp.reshape(inp.shape[0], 28, 28, 1)
    inp=np.array(inp, dtype=np.float64)

    inp /= 255
    prediction = model.predict(inp)
    display_image(img_patch_short)
    
    #l1.append(img_patch_short)
    #l2.append(decode(input()))
    
    preds_classes = np.argmax(prediction, axis=-1)
    print(encode(preds_classes))
    

White Rook
Blank Square
White Pawn
Blank Square
Blank Square
Blank Square
Black Pawn
Black Rook
White Horse
White Pawn
Blank Square
Blank Square
Blank Square
Blank Square
Black Pawn
Black Horse
White Bishop
White Pawn
Blank Square
Blank Square
Black Bishop
Blank Square
Black Pawn
Black Bishop
White King
Blank Square
White Pawn
Blank Square
Blank Square
Blank Square
Black Pawn
Black King
White Queen
Blank Square
White Pawn
Blank Square
Black Pawn
Blank Square
Blank Square
Black Queen
White Bishop
Blank Square
White Pawn
Blank Square
Blank Square
Black Horse
Black Pawn
Blank Square
White Horse
White Pawn
Blank Square
Blank Square
Blank Square
Blank Square
Black Pawn
Blank Square
White Rook
White Pawn
Blank Square
Blank Square
Blank Square
Blank Square
Black Pawn
Black Rook


In [32]:
d1 = np.dstack(l1)
d2 = np.array(l2)

from numpy import save
save('dataset_test_x.npy', d1)
save('dataset_test_y.npy',d2)

ValueError: need at least one array to concatenate